In [3]:
import os
import pandas as pd
import pickle
from preprocessing_type import identity

#exp_name = 'preprocessing_type'
#exp_name = 'in_out_reshape'
exp_name = 'lag_features'
exp_path = os.path.join('logs', exp_name)
results_path = os.path.join(exp_path,'gridsearch_results.pkl')
with open(results_path, 'rb') as f:
    grid_search_results = pickle.load(f)
print(grid_search_results[0].keys())

dict_keys(['dataset-preprocessing_functions', 'input_lag', 'output_lag', 'metrics_per_fold'])


In [5]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

49
dataset-preprocessing_functions: <function identity at 0x00000258B4D3F1A0>
input_lag: 0
output_lag: 0
	Fold 1:
		mean_squared_error: 0.000020 (0.881123)
			per_sample s (u):
				 [1.5989359e-05, 9.100595e-06, 1.6080126e-05, 9.103696e-06, 3.625863e-05, 1.7175671e-05, 9.1842785e-06, 1.8108087e-05, 1.3292001e-05, 1.3167346e-05, 3.299815e-05, 3.5751193e-05, 1.1048948e-05, 1.2465427e-05, 1.8337263e-05, 1.8489558e-05, 1.464033e-05, 1.892965e-05, 9.197582e-06, 1.4675875e-05, 1.1258605e-05, 1.2440874e-05, 9.11978e-06, 1.3370353e-05, 1.261502e-05, 5.473081e-05, 3.658849e-05, 2.072741e-05, 3.7585338e-05, 3.5899313e-05, 3.344216e-05, 9.1308175e-06, 1.7367689e-05, 1.1612181e-05, 1.36466315e-05, 4.1563322e-05, 1.363109e-05, 1.1017933e-05, 3.9948154e-05, 1.0064771e-05, 5.262601e-05, 9.493928e-06, 3.5498193e-05, 1.943522e-05, 9.100782e-06]
				([0.61320853, 0.5807214, 0.6217288, 0.58110684, 1.3771496, 0.5978922, 0.5779043, 0.61420476, 0.6964005, 0.59995025, 1.1375537, 1.324165, 0.8170334, 0.889895

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [32]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re
import functools

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer) or isinstance(exp[key], functools.partial):
                name += exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                name += str(key) + "=" + str(exp[key])

    name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

def filter_experiment(exp):
    return False

def rename_lag_exp_name(exp_name: str):
    if exp_name.startswith("Identity"):
        return "Identity"
    if exp_name.startswith("Lag features"):
        exp_name = exp_name.replace("Lag features ", "")
    exp_name = exp_name.replace("input lag=", "in_lag=").replace("output lag=", "out_lag=")
    return exp_name


exp_samples = {}
for i, exp in enumerate(grid_search_results):
    exp_name = get_exp_name(exp)
    if filter_experiment(exp):
        continue
    exp_name = rename_lag_exp_name(exp_name)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

for exp_name, samples in exp_samples.items():
    print(f"{exp_name}: {len(samples)}")
    print(f"mean: {sum(samples)/len(samples)}")
    print(f"std: {pd.Series(samples).std()}")
    


Identity: 450
mean: 0.9665286586019728
std: 0.3627159297466278
in_lag=0 out_lag=2: 450
mean: 1.0306977186600368
std: 0.48923036456108093
in_lag=0 out_lag=3: 450
mean: 1.045493354929818
std: 0.6741188168525696
in_lag=0 out_lag=4: 450
mean: 1.0137027667628393
std: 0.5694329142570496
in_lag=0 out_lag=6: 450
mean: 0.931536977092425
std: 0.32091155648231506
in_lag=0 out_lag=8: 450
mean: 1.0286746899286905
std: 0.42585644125938416
in_lag=0 out_lag=10: 450
mean: 0.9475056896607081
std: 0.4356842041015625
in_lag=2 out_lag=0: 450
mean: 0.9280123283465703
std: 0.38081619143486023
in_lag=2 out_lag=2: 450
mean: 0.8699110343721178
std: 0.3028079569339752
in_lag=2 out_lag=3: 450
mean: 0.9750343506203757
std: 0.5041139125823975
in_lag=2 out_lag=4: 450
mean: 0.877047348022461
std: 0.33676594495773315
in_lag=2 out_lag=6: 450
mean: 1.0221754600604376
std: 0.5338616967201233
in_lag=2 out_lag=8: 450
mean: 0.9851611672507392
std: 0.418041467666626
in_lag=2 out_lag=10: 450
mean: 0.9056786078876919
std: 0.37

In [33]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(49, 450)


FriedmanchisquareResult(statistic=960.3617088435421, pvalue=5.5104965852679925e-170)

In [34]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[3.55045307e-41 2.73594582e-37 3.22896021e-35 ... 9.91472877e-01
 9.98410168e-01 9.99277349e-01]
3.550453070276673e-41 is st. significant
2.735945818661661e-37 is st. significant
3.2289602066909007e-35 is st. significant
7.909576160124922e-35 is st. significant
1.214864416632631e-33 is st. significant
2.996588204315002e-32 is st. significant
1.4650037876177212e-31 is st. significant
2.1585409270770964e-31 is st. significant
2.3701894141846645e-31 is st. significant
3.123523904782901e-31 is st. significant
1.1149040405329634e-29 is st. significant
3.3985768939364806e-29 is st. significant
5.97304737720944e-29 is st. significant
6.480353950957608e-29 is st. significant
9.733653233742941e-29 is st. significant
1.3574651868727437e-28 is st. significant
2.1874213984309115e-28 is st. significant
1.1683962503989794e-27 is st. significant
1.5903186162262011e-26 is st. significant
7.192398104024813e-26 is st. significant
1.3694508794769162e-25 is st. significant
2.1726483030699273e-25 is st. si

(array([ True,  True,  True, ..., False, False, False]),
 array([4.17533281e-38, 3.21473634e-34, 3.79079928e-32, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
 4.3615795731755114e-05,
 4.251700680272109e-05)

In [35]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Identity', 'in_lag=0 out_lag=2', 'in_lag=0 out_lag=3',
       'in_lag=0 out_lag=4', 'in_lag=0 out_lag=6', 'in_lag=0 out_lag=8',
       'in_lag=0 out_lag=10', 'in_lag=2 out_lag=0', 'in_lag=2 out_lag=2',
       'in_lag=2 out_lag=3', 'in_lag=2 out_lag=4', 'in_lag=2 out_lag=6',
       'in_lag=2 out_lag=8', 'in_lag=2 out_lag=10', 'in_lag=3 out_lag=0',
       'in_lag=3 out_lag=2', 'in_lag=3 out_lag=3', 'in_lag=3 out_lag=4',
       'in_lag=3 out_lag=6', 'in_lag=3 out_lag=8', 'in_lag=3 out_lag=10',
       'in_lag=4 out_lag=0', 'in_lag=4 out_lag=2', 'in_lag=4 out_lag=3',
       'in_lag=4 out_lag=4', 'in_lag=4 out_lag=6', 'in_lag=4 out_lag=8',
       'in_lag=4 out_lag=10', 'in_lag=6 out_lag=0', 'in_lag=6 out_lag=2',
       'in_lag=6 out_lag=3', 'in_lag=6 out_lag=4', 'in_lag=6 out_lag=6',
       'in_lag=6 out_lag=8', 'in_lag=6 out_lag=10', 'in_lag=8 out_lag=0',
       'in_lag=8 out_lag=2', 'in_lag=8 out_lag=3', 'in_lag=8 out_lag=4',
       'in_lag=8 out_lag=6', 'in_lag=8 out_lag=8', 'in_l

In [36]:
exp_samples.values()

dict_values([[0.7830754, 0.7620503, 0.78849787, 0.7623031, 1.1735206, 0.7732353, 0.76020014, 0.78371125, 0.8345069, 0.77456397, 1.0665702, 1.1507221, 0.90389925, 0.94334364, 0.78790164, 0.7918409, 0.84913784, 0.80594885, 0.75826484, 0.84607184, 0.9134585, 0.7345698, 0.7623464, 0.80346394, 0.94604003, 1.716095, 1.1273755, 0.9028512, 1.1021813, 1.1600758, 1.0499848, 0.76196945, 0.7742191, 0.7794116, 0.85749614, 1.0609306, 0.8501376, 0.7701923, 1.038357, 0.72409904, 1.6447041, 0.74946177, 1.1735852, 0.85519254, 0.76182616, 1.1859986, 1.0712293, 0.79361635, 1.4643207, 0.9273135, 1.5007638, 1.4577191, 0.7972986, 0.8944651, 1.4984583, 1.5599166, 1.4420193, 0.9171343, 0.773726, 1.529103, 1.100529, 0.9971814, 0.71441793, 1.5870222, 1.6084422, 1.4678835, 0.8983515, 1.416309, 1.3322912, 1.4524071, 1.4151304, 1.5486395, 1.5637351, 1.4259628, 1.5843134, 1.2122594, 1.0710211, 1.2165936, 1.4557103, 1.0918803, 1.41985, 1.1253457, 1.07751, 1.0669667, 1.5807494, 1.070932, 0.77771586, 0.80178547, 0.7218

In [37]:
sequences = "whole" # "whole", "last", "all"

chosen_samples = {}
if sequences == "whole":
    for key, value in exp_samples.items():
        if not "not" in key:
            key = key.replace("return sequences", "")
            chosen_samples[key] = value
elif sequences == "last":
    for key, value in exp_samples.items():
        if "not" in key:
            key = key.replace("return sequences", "")
            chosen_samples[key] = value
else:
    chosen_samples = exp_samples

x=np.array(list(chosen_samples.values()))
treatment_names = list(chosen_samples.keys())
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

in_lag=10 out_lag=6: 19.057777777777776
in_lag=3 out_lag=10: 19.753333333333334
in_lag=10 out_lag=8: 20.09777777777778
in_lag=2 out_lag=2: 20.42888888888889
in_lag=2 out_lag=4: 20.535555555555554
in_lag=8 out_lag=8: 21.8
in_lag=3 out_lag=3: 21.924444444444443
in_lag=8 out_lag=2: 22.413333333333334
in_lag=2 out_lag=10: 22.675555555555555
in_lag=3 out_lag=0: 22.704444444444444
in_lag=4 out_lag=8: 22.906666666666666
in_lag=3 out_lag=2: 23.284444444444443
in_lag=6 out_lag=8: 23.29111111111111
in_lag=10 out_lag=0: 23.484444444444446
in_lag=4 out_lag=3: 23.6
in_lag=0 out_lag=3: 23.653333333333332
in_lag=8 out_lag=3: 23.704444444444444
in_lag=3 out_lag=6: 23.82
in_lag=6 out_lag=3: 23.86888888888889
in_lag=2 out_lag=3: 23.91777777777778
in_lag=3 out_lag=4: 23.95111111111111
in_lag=4 out_lag=0: 23.99111111111111
in_lag=0 out_lag=4: 24.15111111111111
in_lag=4 out_lag=6: 24.295555555555556
in_lag=2 out_lag=6: 24.497777777777777
in_lag=0 out_lag=10: 24.655555555555555
in_lag=8 out_lag=10: 24.66666

In [38]:
#groups = diagram.get_groups(alpha=0.05, adjustment="holm", return_names=True)
#print("Groups:", groups)

#print(diagram.average_ranks)
#diagram.get_groups(alpha=0.05, adjustment="holm")
diagram.to_file(
    f"critdd_io_reshape_whole_lags.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
    #axis_options = {"title", "Normalization methods"}
)